In [2]:
import unittest
import warnings
import numpy as np
import logging as log
import subprocess as sp
import random as random
import parmed as pmd
import paprika
from paprika.openmm_simulate import *
from paprika.restraints import *
from paprika.utils import HAS_OPENMM, decompose_openmm_energy

%load_ext autoreload
%autoreload 2

2018-02-06 11:41:59 AM OpenMM support: Yes


In [3]:
my_simulation = OpenMM_GB_simulation()
my_simulation.topology = '../test/cb6-but/vac.topo'
my_simulation.md['temperature'] = 300
my_simulation.md['nonbonded_method'] = app.NoCutoff
my_simulation.md['friction'] = 1/unit.picosecond
my_simulation.md['timestep'] = 0.0005*unit.picoseconds
my_simulation.md['constraints'] = None
my_simulation.md['platform'] = 'Reference'
my_simulation.md['precision'] = 'double'
my_simulation.md['coordinates'] = '../test/cb6-but/vac.crds'
my_simulation.md['steps'] = 10
my_simulation.md['output'] = 'tmp.nc'

In [142]:
forces = []
for i in [0, 1]:
    prmtop = app.AmberPrmtopFile('../test/cb6-but/vac.topo')
    inpcrd = app.AmberInpcrdFile('../test/cb6-but/vac.crds')
#     system = prmtop.createSystem(nonbondedMethod=app.NoCutoff, constraints=None)
#     integrator = mm.LangevinIntegrator(300, 1/unit.picosecond,  0.0005*unit.picoseconds)
#     simulation = app.Simulation(prmtop.topology, system, integrator, mm.Platform.getPlatformByName('Reference'))
    
    system = my_simulation.setup_system(my_simulation.md, seed=42)    
    
    if i:
        print('Adding restraints...')
        bond_restraint = mm.CustomBondForce('k * (r - r_0)^2')
        bond_restraint.addPerBondParameter('k')
        bond_restraint.addPerBondParameter('r_0')

        r_0 = 10.5 * unit.nanometers
        k = 1000 * unit.kilojoules_per_mole / unit.nanometers ** 2
        bond_restraint.addBond(1, 100, [k, r_0])
        system.addForce(bond_restraint)

    simulation = my_simulation.setup_simulation(system, my_simulation.md)
        
    # simulation.context.setPositions(inpcrd.positions)
    simulation.context.setVelocitiesToTemperature(300 * unit.kelvin, 42)
    simulation.step(1000)
    
    omm_f = simulation.context.getState(getForces=True).getForces(asNumpy=True).value_in_unit(unit.kilocalories_per_mole/unit.angstroms)
    forces.append(omm_f)

    state = simulation.context.getState(getEnergy=True)
    energy = state.getPotentialEnergy() / unit.kilocalories_per_mole
    print(energy)

[autoreload of paprika.restraints failed: Traceback (most recent call last):
  File "/home/dslochower/data/applications/anaconda3/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 247, in check
    superreload(m, reload, self.old_objects)
ValueError: __init__() requires a code object with 1 free vars, not 0
]
2018-02-01 08:17:33 PM Integrator random number seed: 42
2018-02-01 08:17:33 PM Platform: Reference
2018-02-01 08:17:44 PM Integrator random number seed: 42
2018-02-01 08:17:44 PM Platform: Reference


-729.419987123009
Adding restraints...
22847.43843452656


In [61]:
simulation.system

<simtk.openmm.openmm.System; proxy of <Swig Object of type 'OpenMM::System *' at 0x7fe633b751b0> >

In [3]:
for restraint_on in [0, 1]:
    energies = []
    prmtop = app.AmberPrmtopFile('../test/cb6-but/vac.topo')
    inpcrd = app.AmberInpcrdFile('../test/cb6-but/vac.crds')
    system = prmtop.createSystem(nonbondedMethod=app.NoCutoff, constraints=None)
    integrator = mm.LangevinIntegrator(300, 1/unit.picosecond,  0.0005*unit.picoseconds)

    for force in system.getForces():
        force.setForceGroup(10)
    
    if restraint_on:
        bond_restraint = mm.CustomBondForce('k * (r - r_0)^2')
        bond_restraint.addPerBondParameter('k')
        bond_restraint.addPerBondParameter('r_0')

        r_0 = 10.5 * unit.nanometers
        k = 1000 * unit.kilojoules_per_mole / unit.nanometers ** 2
        bond_restraint.addBond(1, 100, [k, r_0])
        system.addForce(bond_restraint)
        bond_restraint.setForceGroup(1)

    simulation = app.Simulation(prmtop.topology, system, integrator, mm.Platform.getPlatformByName('Reference'))   

    simulation.context.setPositions(inpcrd.positions)
    simulation.context.setVelocitiesToTemperature(300 * unit.kelvin, 42)
    simulation.step(10)
    
    total_state = simulation.context.getState(getEnergy=True)
    total_energy = total_state.getPotentialEnergy() / unit.kilocalories_per_mole
    
    if restraint_on:
        restraint_state = simulation.context.getState(getEnergy=True, groups={1})
        restraint_energy = restraint_state.getPotentialEnergy() / unit.kilocalories_per_mole
        
        non_restraint_state = simulation.context.getState(getEnergy=True, groups={10})
        non_restraint_energy = non_restraint_state.getPotentialEnergy() / unit.kilocalorie_per_mole
    
    print('Total energy = {0:4.4f}'.format(total_energy))
    if restraint_on:
        print('Restraint energy = {0:4.4f}'.format(restraint_energy))
        print('Non-restraint energy = {0:4.4f}'.format(non_restraint_energy))

Total energy = -641.2481
Total energy = 23353.1320
Restraint energy = 23931.3301
Non-restraint energy = -578.1982


In [15]:
tmp = simulation.context.getState(getEnergyC

In [16]:
tmp.getEnergyParameterDerivatives()

TypeError: Parameter derivatives were not requested in getState() call, so are not available.

In [146]:
simulation.topology

<Topology; 1 chains, 2 residues, 122 atoms, 139 bonds>

In [128]:
pmd.openmm.utils.energy_decomposition(prmtop, simulation.context)

KeyError: 0

In [125]:
parm = pmd.amber.AmberParm('../test/cb6-but/vac.topo')

In [127]:
parm

<AmberParm 122 atoms; 2 residues; 139 bonds; parametrized>

In [129]:
pmd.openmm.utils.energy_decomposition(parm, simulation.context)

{'bond': -641.6542515382795, 'total': -641.6542515382795}

In [17]:
import paprika
from paprika.align import *
from paprika.restraints import *
from paprika.utils import *

In [22]:
index_from_mask

<function paprika.utils.index_from_mask>